# Web Scraping - LinkedIn Data

LinkedIn is a very rich source of content when it comes to employment or various types of opportunities. Today we will be scraping some data off LinkedIn based on some keywords or a particular hashtag.<br>
Let's start with taking input the keywords or a hastag to search.

# Creating an Evironment
Importing the packages and libraries needed

In [1]:
import os, random, sys, time
from datetime import datetime
from urllib.parse import urlparse
from selenium import webdriver 
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Since LinkedIn only allows a single hashtag search, we will take input for only one hashtag. However we may search for multiple keywords, so we will take multiple inputs for a keyword based search 

In [3]:
print('Enter 1 to search for multiple keywords')
print('Enter 2 to search by a single hashtag')
s = -1
n = int(input())
if n==1:
    print('Enter keywords separated by space')
    keywords = input()
    s = 1
if n==2:
    print('Enter a hashtag')
    keywords = input()
    s = 2

Enter 1 to search for multiple keywords
Enter 2 to search by a single hashtag
1
Enter keywords separated by space
decodr-technologies


In [4]:
# Start up the script web-browser

In [5]:
browser = webdriver.Chrome('chromedriver.exe')

In [6]:
# Open the login link from the browser
browser.get('https://www.linkedin.com/uas/login')

Create a config txt file, The first line of this file will be the LinkedIn username, second line will be the password.

In [14]:
# Getting the username and password from config file
file = open('config.txt')
lines = file.readlines()
username = lines[0]
password = lines[1]

In [15]:
# Sending credentials to LinkedIn login page
elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

In [16]:
elementID = browser.find_element_by_id('password')
elementID.send_keys(password)

# Generate the url
This is done to search for the hashtag or keyword. After creating the url, send it to the browser.

In [18]:
#link = 'https://www.linkedin.com/feed/hashtag/sample/'
#link = 'https://www.linkedin.com/search/results/content/?keywords=research%2C%20intern'
content = [i for i in keywords.strip().split(' ')]
if s == 1:
    link = 'https://www.linkedin.com/search/results/content/?keywords='
    for jj in range(len(content)):
        if jj != len(content)-1:
            link = link + content[jj] + '%2C%20'
        if jj == len(content)-1:
            link = link + content[jj]
if s == 2:
    link = 'https://www.linkedin.com/feed/hashtag/'
    link = link + content[0] + '/'

browser.get(link)

Once the url is entered, scroll to the required amount of page length to get maximum data

In [19]:
SCROLL_PAUSE_TIME = 3
# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")
for i in range(10):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
browser.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)

LinkedIn condenses its posts by showing a small preview. We must press on all the "see more" buttons to expand the content and reveal the full data of all posts.

In [22]:
# Press "See more" button for all the visible posts
browser.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)
connect_buttons = browser.find_elements_by_class_name('feed-shared-inline-show-more-text__see-more-less-toggle')
for connect_button in connect_buttons:
    #WebDriverWait(browser, 20).until(EC.element_to_be_clickable(connect_buttons)).click()
    connect_button.click()
    print(connect_button)

<selenium.webdriver.remote.webelement.WebElement (session="8bef617dd36395d6b429f1e9b3ef3c62", element="6bb25d38-d665-4a57-b456-bca485d59839")>
<selenium.webdriver.remote.webelement.WebElement (session="8bef617dd36395d6b429f1e9b3ef3c62", element="59641196-94bd-4271-8791-4eca1041e25c")>
<selenium.webdriver.remote.webelement.WebElement (session="8bef617dd36395d6b429f1e9b3ef3c62", element="cb626c17-a901-4a65-971a-2d29e853f2ef")>
<selenium.webdriver.remote.webelement.WebElement (session="8bef617dd36395d6b429f1e9b3ef3c62", element="9e3e6f6c-0724-4162-9e13-297681b6489a")>
<selenium.webdriver.remote.webelement.WebElement (session="8bef617dd36395d6b429f1e9b3ef3c62", element="b5bb4c44-d2ae-4028-a2fb-7884f3a6cb1d")>
<selenium.webdriver.remote.webelement.WebElement (session="8bef617dd36395d6b429f1e9b3ef3c62", element="fbf869ad-2232-4684-ac2a-5d55addd000b")>
<selenium.webdriver.remote.webelement.WebElement (session="8bef617dd36395d6b429f1e9b3ef3c62", element="c38dff03-d3cf-45cb-b0a9-1de6991e2c97")>

Now the data is ready to be scraped. We will use beautiful soup to extract the data and then, clean it.

In [24]:
timestamp = datetime.now()
src = browser.page_source
soup = BeautifulSoup(src, 'lxml')

In [25]:
master_data = soup.find_all('div', {'class': 'feed-shared-update-v2'})
info = soup.find_all('div', {'class':'feed-shared-actor__meta'})

Search for the data we need by its class name. Then  run a for loop to get the Name of the person who posted,their position and designation and content of the post. We also exctract the links, and take content of a post that is shared in another post.

In [26]:
a = []
for i in range(len(master_data)):
    for j in range(10):
        try:
            Name = master_data[i].find_all('div', {'class': 'feed-shared-actor__meta'})[j].find('span', {'dir':'ltr'}).get_text().strip()
            Post = master_data[i].find_all('span', {'class': 'feed-shared-actor__description'})[j].get_text().strip()
            Content = master_data[i].find_all('div', {'class': 'feed-shared-text__text-view'})[j].get_text().strip()
            print(Name)
            a.append(Name)
            print(Post)
            a.append(Post)
            print(Content)
            a.append(Content)
            print('TAGGED-------------')
            a.append('Same post, shared post:')
            try:
                links = master_data[i].find('a', {'class': 'feed-shared-article__meta'}).get('href')
            except:
                print(links)
        except:
            pass
    a.append(' ')
    a.append(' ')
    print('================================LINE BREAK============================================')

Sam Baisla
CEO of Startup India Foundation | Co-Founder of Brand Samosa, DECODR & NEXEL | Hiring for Global Positions
We just launched a startup successfully amidst the deadliest pandemic of the past 100 years. 

While others are shutting shops and firing employees, we just hired 200+ interns and 4 full-time employees and decided to expand globally at Brand Samosa

The whole world is under lockdown, and Decodr Technologies achieved operations profitability within the first month itself.

Now, you may see some magic here, but the preparation has been on for months...even years.

Did I know about the pandemic beforehand?

Absolutely not.

But we were preparing...not for the pandemic, but for the future.

If you want to know how we are able to do it...I might share a thing or two.

Let me know in the comments section what you think are the ways...a startup can thrive through the uncertain times like these.

It would be interesting to learn from you.

Jyoti Aery Rajat Kumar Anirban Chakrab

================================LINE BREAK============================================
Abhishek K.
Looking for Job Opportunity in Data Science & Machine Learning
Recently accepted Offer letter for Product Research Intern of Decodr Technologies.
TAGGED-------------
https://docs.google.com/forms/d/e/1FAIpQLSfLcDK5C5rl2qkUloxKzFkSxxnwF2Nj8AuBMZOX4VG_lt31JQ/viewform?usp=send_form&usp=embed_facebook
================================LINE BREAK============================================
Subhadip Samanta
Summer Intern (Trade) at Spices Board India || IIFT Delhi || Ex-Infosys || 10 B-School Competitions Winner & Finalist
I have developed my first Machine Learning exhaustive project as a part of Data Analytics & Machine Learning internship at Decodr Technologies. 

The project is about predicting a financial organization will give loan to its customers or not on the basis of some other features of the customers.

I used 4 different models to train my data and compared them
a) logistic regression

================================LINE BREAK============================================
Tushar Srivastava
Digital Marketing Intern at Allied Market Research
In today’s era we see lots of area where Machine learning is involved and helping in our daily life such as:

⭕ The heavily hyped, self-driving Google car? The essence of machine learning. 🚗

⭕ Online recommendation offers such as those from Amazon and Netflix? Machine learning applications for everyday life.🖥️

⭕ Knowing what customers are saying about you on Twitter? Machine learning combined with linguistic rule creation.

⭕ Fraud detection? One of the more obvious, important uses in our world today.💰

Feel free to share where you think machine learning is involved.

hashtag#decodr hashtag#machine_learning hashtag#data hashtag#technology hashtag#future
TAGGED-------------
https://github.com/subhadip817/Decodr-Machine-Learning-Project/tree/Loan-Prediction-Model
================================LINE BREAK============================

# Save Data
As you can see, we have successfully extracted all the data and saved it into a list. We now organise it and save it into a csv file. Since data is scrapped at regular periods of interval to stay upto date with the content, we save the csv name as the current timestamp to keep track of the scraped data.

In [27]:
svst = timestamp.strftime("%d/%m/%Y %H:%M:%S")
save_name ='LinkedIn_Data_Scrape_' + svst + '.csv'
save_name = save_name.replace(" ", "_")
save_name = save_name.replace("/", "-")
save_name = save_name.replace(":", "-")
print(save_name)
np.savetxt(save_name,a, fmt = '%s', encoding='utf-8')

LinkedIn_Data_Scrape_07-05-2020_01-03-35.csv


# Conclusion

Web Scraping is an extremely powerful technique to get huge amounts of data in a couple of seconds. It acts as a great tool for Data Scientists and Analysis to get hold of data and process it for various applications. Pls note, scrapping data may not be legal for all websites, make sure that you read about the licenses and notices of a website before scraping off any data.
Good Luck!